# 맛집 유튜브 채널 찾기
결과 : data/youtube_channel.xlsx  

방법1 : 목록을 정리해 놓은 글을 찾아서 정리한다.  
방법2 : [PLAYBOARD](https://playboard.co/)에서 특정 검색어(부산 맛집)를 입력해 크롤링한다.   
방법3 : 유튜브에서 특정 검색어(맛집)를 입력해 크롤링한다.  

# 유튜브 채널에서 추천한 맛집 찾기
~~방법1 : 유튜브API : 동영상은 제공해주지만 동영상 제목은 제공해주지 않습니다.~~  
방법 : 유튜브 크롤링

In [1]:
# 사용한 라이브러리
from selenium import webdriver
import time
import pandas as pd

In [2]:
# 유튜브 채널 가져오기
youtube_channels = pd.read_excel("./data/youtube_channel.xlsx", sheet_name="Sheet1")
# '제공'열 삭제
youtube_channels.drop('제공', axis=1, inplace=True)
# 중복 데이터 삭제
youtube_channels.drop_duplicates(keep='first')
display(youtube_channels.head(5))

,크리에이터
0,맛상무
1,정육왕
2,맛있겠다 Yummy
3,섬마을훈태TV
4,떡볶퀸 Tteokbokqueen


In [ ]:
# 크롤링

# 드라이버 준비
driver = webdriver.Chrome('C:/Temp/chromedriver')
# 유튜브 들어가기
driver.get("https://www.youtube.com/c/"+ youtube_channels[1,1] +"/videos?view=0&sort=p&flow=grid")
time.sleep(2)

In [14]:
sorting = driver.find_element_by_css_selector ('#icon-label')
sorting.click()
time.sleep(3)

sortway = driver.find_element_by_css_selector ('contenWrapper#div a.yt-simple-endpoint')
sortway.click()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"contenWrapper#div a.yt-simple-endpoint"}
  (Session info: chrome=94.0.4606.81)


In [ ]:
https://www.youtube.com/channel/UCxGyH7vzYkZwFseTUa3yZpQ/videos?view=0&sort=p